In [19]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# ml-100k: 10만 개 영화 평점 데이터
data = Dataset.load_builtin('ml-100k')

# surprise의 train_test_split() 사용. trainset : testset = 3 : 1
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [32]:
# Surprise Dataset -> Pandas DataFrame
pd_data = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','movie_id','rating','timestamp'])
pd_data

,user_id,movie_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [20]:
# SVD를 trainset에 적용
algo = SVD(n_epochs=20, n_factors=50, random_state=0) # GridSearchCV를 이용한 최적 하이퍼 파라미터 적용
algo.fit(trainset)

In [21]:
# 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력
uid = str(196)
iid = str(302)

# 추천 예측 평점 (.predict)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.37904004664696, details={'was_impossible': False})

In [22]:
# 추천 예측 평점 (.test)
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')

predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6532744639165786, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.9097372780826998, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.8776919127707177, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.704888702614823, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3572494565126663, details={'was_impossible': False})]

In [23]:
# 속성 확인
[ (pred.uid, pred.iid, pred.est, pred.details) for pred in predictions[:3] ]

[('120', '282', 3.6532744639165786, {'was_impossible': False}),
 ('882', '291', 3.9097372780826998, {'was_impossible': False}),
 ('535', '507', 3.8776919127707177, {'was_impossible': False})]

In [24]:
# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.9457


0.9456994032241722

In [18]:
# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) # algo가 아닌 SVD 입력하였다.
gs.fit(data)

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

{'n_epochs': 20, 'n_factors': 50}
0.9429102174553406


In [70]:
# 아직 보지 않은 영화 리스트 함수
def get_unseen_surprise(movies, user_id):
     # 특정 user_id가 평점을 매긴 모든 영화 리스트
    seen_movies = movies.loc[pd_data['user_id'] == str(user_id)].values.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    total_movies = movies['movie_id'].drop_duplicates()
      
    # 한줄 for + if문으로 안 본 영화 리스트 생성
    unseen_movies = [ movie for movie in total_movies if movie not in seen_movies]
    
    # 일부 정보 출력
    total_movie_cnt = len(total_movies)
    seen_cnt = len(seen_movies)
    unseen_cnt = len(unseen_movies)
    
    print(f"전체 영화 수: {total_movie_cnt}, 평점 매긴 영화 수: {seen_cnt}, 추천 대상 영화 수: {unseen_cnt}")
    
    return unseen_movies

In [71]:
unseen_movies = get_unseen_surprise(pd_data, 326)
unseen_movies

전체 영화 수: 1682, 평점 매긴 영화 수: 187, 추천 대상 영화 수: 1682


['242',
 '302',
 '377',
 '51',
 '346',
 '474',
 '265',
 '465',
 '451',
 '86',
 '257',
 '1014',
 '222',
 '40',
 '29',
 '785',
 '387',
 '274',
 '1042',
 '1184',
 '392',
 '486',
 '144',
 '118',
 '1',
 '546',
 '95',
 '768',
 '277',
 '234',
 '246',
 '98',
 '193',
 '88',
 '194',
 '1081',
 '603',
 '796',
 '32',
 '16',
 '304',
 '979',
 '564',
 '327',
 '201',
 '1137',
 '241',
 '4',
 '332',
 '100',
 '432',
 '322',
 '181',
 '196',
 '679',
 '384',
 '143',
 '423',
 '515',
 '20',
 '288',
 '219',
 '526',
 '919',
 '26',
 '232',
 '427',
 '512',
 '15',
 '1049',
 '416',
 '165',
 '690',
 '248',
 '1444',
 '5',
 '229',
 '237',
 '480',
 '54',
 '366',
 '518',
 '403',
 '111',
 '625',
 '338',
 '25',
 '1016',
 '154',
 '275',
 '1153',
 '498',
 '382',
 '209',
 '23',
 '294',
 '208',
 '685',
 '328',
 '496',
 '132',
 '174',
 '96',
 '151',
 '307',
 '648',
 '21',
 '832',
 '514',
 '789',
 '485',
 '317',
 '195',
 '200',
 '385',
 '750',
 '264',
 '245',
 '135',
 '1147',
 '471',
 '658',
 '140',
 '379',
 '815',
 '479',
 '368

In [79]:
def recomm_movie_by_surprise(algo, user_id, unseen_movies, top_n=10):
    
    # 아직 보지 않은 영화의 예측 평점: prediction 객체 생성
    predictions = []
    for movie_id in unseen_movies:
        predictions.append(algo.predict(str(user_id), str(movie_id)))
    
    # 리스트 내의 prediction 객체의 est를 기준으로 내림차순 정렬
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True) # key에 리스트 내 객체의 정렬 기준을 입력
    
    # 상위 top_n개의 prediction 객체
    top_predictions = predictions[:top_n]
    
    # 영화 아이디, 제목, 예측 평점 출력
    print(f"Top-{top_n} 추천 영화 리스트")
    print(f"movie_id : est_rating")
    
    for pred in top_predictions:
        
        movie_id = pred.iid
        movie_rating = pred.est
        
        print(f"{movie_id} : {movie_rating:.2f}")

In [80]:
recomm_movie_by_surprise(algo, 326, unseen_movies, top_n=20)

Top-20 추천 영화 리스트
movie_id : est_rating
408 : 4.19
318 : 4.09
178 : 4.09
64 : 4.05
189 : 4.00
483 : 3.97
513 : 3.96
132 : 3.95
169 : 3.93
603 : 3.93
79 : 3.93
427 : 3.92
648 : 3.91
251 : 3.87
527 : 3.87
659 : 3.86
520 : 3.86
498 : 3.86
313 : 3.86
615 : 3.85
